<a href="https://colab.research.google.com/github/PaarthBarkur1/Next-Word-Predictor/blob/main/lstmwordgenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import copy
import warnings

https://en.wikipedia.org/wiki/Virat_Kohli you can use this

In [2]:
import requests

url = str(input("Provide the link to the website that needs to be scrapped : "))
response = requests.get(url)
html_content = response.text


Provide the link to the website that needs to be scrapped : https://en.wikipedia.org/wiki/Virat_Kohli


In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content, 'html.parser')


In [4]:
def extract_paragraphs(soup):
    paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]  # Extract text from paragraph tags
    return [p for p in paragraphs if p and len(p) > 10]  # Return paragraphs longer than 10 characters

In [5]:

all_text = ' '.join(extract_paragraphs(soup))
all_text = all_text.replace('\n', ' ')
all_text = all_text.replace('\t', ' ')
all_text

'Virat Kohli(Hindi pronunciation:[ʋɪˈɾɑːʈˈkoːɦli]ⓘ; born 5 November 1988) is an Indianinternational cricketerwho playsTestandODIcricket for theIndian national team. A formercaptainin all formats of the game, Kohli retired from theT20Iformat following India\'s win at the2024 T20 World Cup. He\'s a right-handedbatsmanand an occasional unorthodox right arm quickbowler. In total runs scored, Kohli ranks first inIPL, second inT20I, third inODI, and fourth overall in international cricket.[4]Regarded as one of the greatest batsman of all time, he also holds the record for scoring the most centuries in ODI cricket and is second in thelist of most international centuries scoredin international cricket.[5] Kohli was a member of the Indian team that won the2011 Cricket World Cup,2013 Champions Trophyand2024 T20 World Cupand captained India to win theICC Test macethree consecutive times in 2017, 2018, and 2019.[6]He representsRoyal Challengers Bengaluruin theIndian Premier LeagueandDelhiindomesti

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([all_text])

In [7]:
len(tokenizer.word_index)

1456

In [8]:
input_sequences = []
maxi = 0
for sentences in all_text.split("."):
  if len(sentences) > 1:
    tokenized_sentence = (tokenizer.texts_to_sequences([sentences]))[0]
    for i in range(1, len(tokenized_sentence)):
      maxi = max(len(tokenized_sentence[:i+1]), maxi)
      input_sequences.append(tokenized_sentence[:i+1])


In [9]:
maxi

49

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences =pad_sequences(input_sequences,maxlen = maxi , padding='pre')


In [11]:
x = padded_sequences[:,:-1]
y = padded_sequences[:,-1]


In [12]:
x.shape

(3421, 48)

In [13]:
from tensorflow.keras.utils import to_categorical
y =to_categorical(y, num_classes = (len(tokenizer.word_index)+1))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.optimizers import Adam

# Assuming tokenizer is already defined and fitted
inpsize = len(tokenizer.word_index) + 1
  # Replace with the actual max length of your input sequences

# Model definition
model = Sequential()
model.add(Embedding(inpsize,100))
model.add(LSTM(150))
model.add(Dense(inpsize, activation='softmax'))

# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)
model.fit(x,y,epochs=100,callbacks=[callback])

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0406 - loss: 6.9351
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0525 - loss: 6.2637
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0556 - loss: 6.0109
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0541 - loss: 5.9040
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0686 - loss: 5.6450
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0918 - loss: 5.3806
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0965 - loss: 5.2100
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1243 - loss: 4.9478
Epoch 9/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1224 - loss: 4.8326
Epoch 10/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1417 - loss: 4.5712
Epoch 11/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1565 - loss: 4.3889
Epoch 12/100
107/107 ━━━━━━━━━━

In [25]:
import time
text = "Kohli's playstyle is"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=maxi, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Kohli's playstyle is since
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Kohli's playstyle is since then
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Kohli's playstyle is since then has
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Kohli's playstyle is since then has attracted
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Kohli's playstyle is since then has attracted significant
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Kohli's playstyle is since then has attracted significant media
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Kohli's playstyle is since then has attracted significant media interest
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Kohli's playstyle is since then has attracted significant media interest at
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Kohli's playstyle is since then has attracted significant media interest at launch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Kohli's playstyle is since then has attracted significant media interest at launch a
